In [1]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


Loading configurations from /workspaces/fundamentals-of-data-transformation-4554074/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaylimit,10
displaycon,False


,Count
0,224


Build a query to fetch all the parks in Utah (UT) and order the results by the park name.

In [2]:
%%sql

SELECT 
    fullname, 
    split(states, ',') ::string[] AS states_list
from nps_public_data.parks p
WHERE list_has_any(states_list, ['UT'])
ORDER by fullname asc;

-- done

,fullName,states_list


Build a query to fetch all the National Parks that cross state boundaries (i.e. parks that are in more than one state). Remember to filter `parks` for *only* National Parks.

Hint: `parks.states` is a string representation of a list, i.e. `UT,CA,NC`. The `parks` table includes parks that aren't National Parks.

In [3]:
%%sql


    SELECT 
        fullname, designation,
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where p.designation ILIKE '%National Park%' and states_count > 1


    -- done


,fullName,designation,states_list,states_count
0,Yellowstone National Park,National Park,"[ID, MT, WY]",3
1,Death Valley National Park,National Park,"[CA, NV]",2
2,Great Smoky Mountains National Park,National Park,"[NC, TN]",2


For all national parks, return the `states` column as a `STRUCT` type with each element as a state.

In [ ]:
%%sql

SELECT
    fullname,
    designation,
    states_list,
    STRUCT_PACK(
        state1 := states_list[1],
        state2 := states_list[2],
        state3 := states_list[3]
    ) AS states_struct
FROM (
    SELECT
        fullname,
        designation,
        split(states, ',')::STRING[] AS states_list
    FROM nps_public_data.parks
    WHERE designation ILIKE '%National Park%'
)
ORDER BY fullname ASC;

-- done
-- duda como se hace struck


,fullName,designation,states_list,states_struct
0,Acadia National Park,National Park,[ME],"{'state1': 'ME', 'state2': None, 'state3': None}"
1,Arches National Park,National Park,[UT],"{'state1': 'UT', 'state2': None, 'state3': None}"
2,Badlands National Park,National Park,[SD],"{'state1': 'SD', 'state2': None, 'state3': None}"
3,Big Bend National Park,National Park,[TX],"{'state1': 'TX', 'state2': None, 'state3': None}"
4,Biscayne National Park,National Park,[FL],"{'state1': 'FL', 'state2': None, 'state3': None}"
...,...,...,...,...
56,Wind Cave National Park,National Park,[SD],"{'state1': 'SD', 'state2': None, 'state3': None}"
57,Wrangell - St Elias National Park & Preserve,National Park & Preserve,[AK],"{'state1': 'AK', 'state2': None, 'state3': None}"
58,Yellowstone National Park,National Park,"[ID, MT, WY]","{'state1': 'ID', 'state2': 'MT', 'state3': 'WY'}"
59,Yosemite National Park,National Park,[CA],"{'state1': 'CA', 'state2': None, 'state3': None}"


Which parks are in **either** Montana (MT) or Wyoming (WY)?

In [ ]:
%%sql

SELECT 
    fullname, 
    split(states, ',') ::string[] AS states_list
from nps_public_data.parks p
WHERE list_has_any(states_list, ['MT', 'WY'])
ORDER by fullname asc;

-- done

,fullName,states_list
0,Big Hole National Battlefield,[MT]
1,Bighorn Canyon National Recreation Area,"[MT, WY]"
2,California National Historic Trail,"[CA, CO, ID, KS, MO, NE, NV, OR, UT, WY]"
3,Devils Tower National Monument,[WY]
4,Fort Laramie National Historic Site,[WY]
5,Fort Union Trading Post National Historic Site,"[MT, ND]"
6,Fossil Butte National Monument,[WY]
7,Glacier National Park,[MT]
8,Grand Teton National Park,[WY]
9,Grant-Kohrs Ranch National Historic Site,[MT]


What about _both_ Montana _and_ Wyoming?

In [13]:
%%sql

SELECT 
    fullname, 
    split(states, ',') ::string[] AS states_list
from nps_public_data.parks p
WHERE list_contains(states_list, 'MT') and list_contains(states_list, 'WY')
ORDER by fullname asc;

,fullName,states_list
0,Bighorn Canyon National Recreation Area,"[MT, WY]"
1,Yellowstone National Park,"[ID, MT, WY]"


Which park is in the greatest number of states? How many states>

In [20]:
%%sql

    SELECT 
        fullname, designation,
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    order by states_count desc, fullname asc
    limit 5;




,fullName,designation,states_list,states_count
0,Lewis & Clark National Historic Trail,National Historic Trail,"[IA, ID, IL, IN, KS, KY, MO, MT, NE, ND, OH, O...",16
1,Appalachian National Scenic Trail,National Scenic Trail,"[CT, GA, MA, MD, ME, NC, NH, NJ, NY, PA, TN, V...",14
2,California National Historic Trail,National Historic Trail,"[CA, CO, ID, KS, MO, NE, NV, OR, UT, WY]",10
3,Washington-Rochambeau Revolutionary Route Nati...,National Historic Trail,"[MA, RI, CT, NY, NJ, PA, DE, MD, VA, DC]",10
4,Trail Of Tears National Historic Trail,National Historic Trail,"[AL, AR, GA, IL, KY, MO, NC, OK, TN]",9


Now, how many parks are in each "group" of state border-crossings? i.e. how many parks are in 1 state, 2 states, 3 states, etc.

Hint: we're grouping by the _number_ of states.

In [28]:
%%sql

with states_list as (
    SELECT 
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
)

select 
  states_count,
  count(*) as parks_count
from states_list
group by states_count
order by states_count desc

    
  


,states_count,parks_count
0,16,1
1,14,1
2,10,2
3,9,1
4,8,2
5,7,3
6,6,2
7,5,2
8,4,4
9,3,10


What's the percentage share of the total? i.e. what *percentage* of parks are in 1 state, 2 states, etc

Hint: window functions might be helpful.

In [ ]:
%%sql

-- pendiente

with states_list as (
    SELECT 
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
)

select 
states_count,
  count(*) over (partition by states_count order by states_count desc) as parks_count,
count(*) / sum(*) over (partition by states_count order by states_count desc) as parks_count

from states_list


BinderException: Binder Error: No function matches the given name and argument types 'sum()'. You might need to add explicit type casts.
	Candidate functions:
	sum(DECIMAL) -> DECIMAL
	sum(SMALLINT) -> HUGEINT
	sum(INTEGER) -> HUGEINT
	sum(BIGINT) -> HUGEINT
	sum(HUGEINT) -> HUGEINT
	sum(DOUBLE) -> DOUBLE

LINE 11: count(*) / sum(*) over (partition by states_count order by states_count desc) as parks_count

from states_list...
                    ^

Write a query that returns the _largest_ campsite in each park. 

As a bonus, do this _without_ using a CTE (filter in the same query)

In [ ]:
%%sql

-- esta a nivel de campgrounds
-- done

SELECT 
    p.fullname as park_name,
    FIRST(c.name) OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as max_num_campsites_site,
         ROW_NUMBER() OVER (PARTITION BY park_name ORDER BY c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable DESC) as park_campsites_rank,
        MAX(c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable) OVER (PARTITION BY park_name) as max_num_campsites,

FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
QUALIFY park_campsites_rank = 1

ORDER BY park_name


,park_name,max_num_campsites_site,park_campsites_rank,max_num_campsites
0,Acadia National Park,Blackwoods Campground,1,281
1,American Memorial Park,American Memorial Park,1,0
2,Amistad National Recreation Area,San Pedro Campground,1,41
3,Arches National Park,Devils Garden Campground,1,51
4,Assateague Island National Seashore,Bayside Drive-in Campground,1,22
...,...,...,...,...
113,Wind Cave National Park,Elk Mountain Campground,1,124
114,Wrangell - St Elias National Park & Preserve,Kendesnii Campground,1,10
115,Yellowstone National Park,Bridge Bay Campground,1,432
116,Yosemite National Park,Tuolumne Meadows Campground,1,304


In [59]:
%%sql

-- List all tables with schema information
SELECT * FROM information_schema.tables WHERE table_schema NOT IN ('information_schema', 'pg_catalog') LIMIT 2;

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,nps_public_data,activities,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None,None
1,memory,nps_public_data,activities__parks,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,None,None


Say you'll be in California (CA) this spring and have time for three National Parks visits. 

How many combinations of national parks can you visit? Can you return the combinations in a list ordered by the name of the first park?

Hint: self-joins and cross joins might be useful.

In [ ]:
%%sql

--done

with california_parks as (
    SELECT 
        name, designation,
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where p.designation ILIKE '%National Park%' and list_contains(states_list, 'CA')
)

select c1.name as park1_name,
       c2.name as park2_name,
       c3.name as park3_name

from california_parks as c1
cross join california_parks as c2
cross join california_parks as c3
where c1.name != c3.name and c2.name != c3.name and c1.name != c2.name

,park1_name,park2_name,park3_name
0,Joshua Tree,Channel Islands,Death Valley
1,Lassen Volcanic,Channel Islands,Death Valley
2,Sequoia & Kings Canyon,Channel Islands,Death Valley
3,Pinnacles,Channel Islands,Death Valley
4,Yosemite,Channel Islands,Death Valley
...,...,...,...
205,Channel Islands,Yosemite,Pinnacles
206,Death Valley,Yosemite,Pinnacles
207,Joshua Tree,Yosemite,Pinnacles
208,Lassen Volcanic,Yosemite,Pinnacles


For our final exercise— find the combinations in alphabetical order, that is, the first letter of each visit occurs in the order of the alphabet, e.g. `[C]hannel Islands, [D]eath Valley, [J]oshua Tree` would satisfy that condition.

Hint: how can you compare strings for alphabetical order? (casing matters)

In [71]:
%%sql

with california_parks as (
    SELECT 
        name, designation,
        split(states, ',') ::string[] AS states_list,
        len(states_list) as states_count
    from nps_public_data.parks p
    where p.designation ILIKE '%National Park%' and list_contains(states_list, 'CA')
)

select c1.name as park1_name,
       c2.name as park2_name,
       c3.name as park3_name

from california_parks as c1
cross join california_parks as c2
cross join california_parks as c3
where c1.name != c3.name and c2.name != c3.name and c1.name != c2.name
and c1.name < c2.name and c1.name < c3.name and c2.name < c3.name
order by c1.name, c2.name, c3.name


,park1_name,park2_name,park3_name
0,Channel Islands,Death Valley,Joshua Tree
1,Channel Islands,Death Valley,Lassen Volcanic
2,Channel Islands,Death Valley,Pinnacles
3,Channel Islands,Death Valley,Sequoia & Kings Canyon
4,Channel Islands,Death Valley,Yosemite
5,Channel Islands,Joshua Tree,Lassen Volcanic
6,Channel Islands,Joshua Tree,Pinnacles
7,Channel Islands,Joshua Tree,Sequoia & Kings Canyon
8,Channel Islands,Joshua Tree,Yosemite
9,Channel Islands,Lassen Volcanic,Pinnacles
